# Imporatations

In [ ]:
!python -m pip install cohere --upgrade


In [1]:
import cohere
API_KEY = 'rY76EgrEdNJQRAWU707DXcpnxWiiEFQzfOEpdISt'
co = cohere.ClientV2(API_KEY)

# Trying out Cohere

In [2]:
import cohere

co = cohere.ClientV2(API_KEY)
response = co.chat(
    model="command-r-plus", 
    messages=[{"role": "user", "content": "hello world!"}]
)

print(response)

id='a6414643-578e-4ab4-bb3d-6f43dfe2730a' finish_reason='COMPLETE' prompt=None message=AssistantMessageResponse(role='assistant', tool_calls=None, tool_plan=None, content=[TextAssistantMessageResponseContentItem(type='text', text='Hello! How can I help you today?')], citations=None) usage=Usage(billed_units=UsageBilledUnits(input_tokens=3.0, output_tokens=9.0, search_units=None, classifications=None), tokens=UsageTokens(input_tokens=196.0, output_tokens=9.0)) logprobs=None


# Adding a cleaning generation function

In [3]:
def get_cohere_response(content: str):
    API_KEY = 'rY76EgrEdNJQRAWU707DXcpnxWiiEFQzfOEpdISt'
    co = cohere.ClientV2(API_KEY)
   
    messages = [{"role": "user", "content": content}]
    chat_response = co.chat(model="command-r-plus", messages=messages)
    
    if hasattr(chat_response, 'message') and hasattr(chat_response.message, 'content'):
        for item in chat_response.message.content:
            if hasattr(item, 'type') and item.type == 'text':
                return item.text
    return ""

In [ ]:
# Try it out with the simple msg "chahwelek"
response_text = get_cohere_response("hi how are you today!")
print(response_text)


Hello! I'm an AI chatbot, so I don't have feelings, but I'm here to assist you with any questions or tasks you may have. How can I help you today?


In [4]:
from pydantic import BaseModel
from enum import Enum

In [6]:
class Category(Enum):
    BUG = 'bug report'
    TECH = 'technical difficulties '
    FEAT = 'feature requests'
    MISC= 'miscellaneous '

#outtput
class ResultRequest(BaseModel):
    category : Category
    summary: str
    extracted_data: str
    
#input
class SupportRequest(BaseModel):
    id: int
    name: str
    organization: str
    text: str

## Prompting

In [41]:
Prompt0 = "You are a customer service agent working in IT, you receive support requests and provide technical support"
PromptCateg = """"There are 4 categories of a support request: bug report or technical difficulties or feature requests or miscellaneous, tell me which one
 is this one (only name the category): """
promptSummary = "give me a concise summary of this support request: "
promptDataBug = """As the support request in this case is a bug report, extract elements such as error codes, affected features/pages,
and triggering actions from this request only:"""
promptDataTech = """As the support request in this case is a technical difficulty, Extract details such as connection issues or
performance issues, or if the client has any difficulties navigating the platform in general from this request only:"""
promptDataFeat = """As the support request in this case is a feature request, Extract the name of the requested feature and any
context indicating urgency (low, medium, high) or benefits from this request only:  """
promptDataMisc = ""


In [25]:
response_text = get_cohere_response(Prompt0 + PromptCateg + "I am having trouble with the platform, I need a new feature that does such and such")
print(response_text)

Feature request.


### Lilttle strs comparison function for later

In [30]:
from difflib import SequenceMatcher

def most_similar(target, options):
    return max(options, key=lambda option: SequenceMatcher(None, target, option).ratio())

In [45]:
target= "hello world"
strs = ["hello", "world", "helo world", "hello wrld", "goodbye world"]
best_match = most_similar(target, strs)
print(best_match)  

helo world


In [43]:
def process_support_request(support_request: SupportRequest) -> ResultRequest:
    
    # used these below cz didnt want to turn to cosine similarity and embeddings
    L = [str(Category.BUG), str(Category.FEAT), str(Category.TECH), str(Category.MISC)]
    category_map = {
    str(Category.BUG): Category.BUG,
    str(Category.FEAT): Category.FEAT,
    str(Category.TECH): Category.TECH,
    str(Category.MISC): Category.MISC,
}

    categ = get_cohere_response(Prompt0 + PromptCateg + support_request.text)
    category_str = most_similar(categ, L)
    category = category_map[category_str]  # Convert back to Category

    summary = get_cohere_response(Prompt0 + promptSummary + support_request.text )

    extracted_data = '' #had to initialize it cz it wouldnt work otherwise
    
    if category == Category.BUG:
        extracted_data = get_cohere_response(Prompt0 + promptDataBug + support_request.text )
    elif category == Category.TECH:
        extracted_data = get_cohere_response(Prompt0 + promptDataTech + support_request.text )
    elif category == Category.FEAT:
        extracted_data = get_cohere_response(Prompt0 + promptDataFeat + support_request.text )
    elif category == Category.MISC:
        extracted_data = get_cohere_response('')

    return ResultRequest(category=category, summary=summary, extracted_data=extracted_data)

sick

In [ ]:
# Exemplef
support_request = SupportRequest(id=1, name="John Doe", organization="Acme Corp", text="I have a bug in the system.")
result_request = process_support_request(support_request)
print(result_request)

category=<Category.BUG: 'bug report'> summary='A customer has reported a bug in the system. They need assistance in identifying and resolving the issue.' extracted_data='No error codes, affected features/pages, or triggering actions were explicitly mentioned in the bug report. However, the user states that they "have a bug in the system," indicating a potential issue with the software or application as a whole.'
